In [11]:
import numpy as np
np.set_printoptions(precision=2, suppress=True, linewidth=100)
g = 3
s = 12
m = 1
m1 = m+1
n = 2*m + 1
n1 = 2*m1
ne = 6
npe = ne+1
ng1 = npe*m1
dl = 1
hi = dl/ne
he = hi
si = 2
bt = 660
c = 4*(s+g)/100/dl**2
c1 = 1
EJ = c1
x = np.arange(hi/2, dl+hi, hi)
q = c*bt*x*(dl-x)
M0 = 2*c*c1
P0 = 0
Ml = -M0
Pl = 0
Dh = np.eye(n1,n1)
for i in range(1, m1):
    ii = i+m1
    Dh[i,i] = hi*Dh[i-1,i-1]
    Dh[ii,ii] = Dh[i,i]
S1 = np.eye(m1,m1)
for j in range(1, m1):
    S1[0,j] = S1[0,j-1] / (j)
for i in range(1, m):
    for j in range(i,m1):
        S1[i,j] = S1[i-1, j-1]
S2 = np.zeros(S1.shape)
S2[m1-1,0] = 1
kk = 0
for i in range(m-1, -1, -1):
    kk = kk+1
    S2[i,i] = S2[i+1,0] / (kk+1)
kk = 0
for i in range(m1-1, -1, -1):
    kk = kk+1
    for j in range(1,m1):
        S2[i,j] = S2[i,j-1] / (j+kk)
invAg = np.eye(n1,n1)
for i in range(m1):
    for j in range(m1):
        invAg[m1+i,j] = S1[i,j]
        invAg[m1+i,m1+j] = S2[i,j]
Ag = np.linalg.inv(invAg)
En = np.eye(n1,n1)
for i in range(2,n1):
    En[i,i] = En[i-1,i-1] / (i)
A0 = np.zeros(En.shape)
for i in range(n1):
    for j in range(n1):
        A0[i,j] = En[i,i]*En[j,j] / (i+j+1)
K0 = Ag.T@A0@Ag
Hv = np.ones((1,n1-1))
H = np.diagflat(Hv,1)
Hvs = np.ones((1,n1-si))
Hs = np.diagflat(Hvs,si)
As = Hs.T@A0@Hs
Ks = Ag.T@As@Ag
b0 = np.eye(n1,1)
for i in range(1,n1):
    b0[i] = b0[i-1]/ (i+1)
bs = b0
Rsz = Ag.T@bs
Ke = EJ/hi**(2*si-1) * (Dh@Ks@Dh) + bt*hi*(Dh@K0@Dh)
KG = np.zeros((ng1,ng1))

for i in range(0,ng1-n,m1):
    for ki in range(n1):
        for kj in range(n1):
            KG[i+ki,i+kj] = KG[i+ki,i+kj] + Ke[ki,kj]
print('Глобальная матрица жесткости KG\n',KG)
Rs = hi * Dh @ Rsz
R0 = np.zeros((n1,1))
R0[0] = P0
R0[1] = M0
Rl = np.zeros((n1,1))
Rl[m1] = Pl
Rl[m1+1] = Ml
RG = np.zeros((ng1, 1))
for i in range(0, ng1-n, m1):
    ie = int(i/m1)
    Ri = q[ie] * Rs
    if ie == 0:
        Ri = Ri + R0
    if ie == ne-1:
        Ri = Ri + Rl
    for ki in range(n1):
        RG[i+ki] = RG[i+ki] + Ri[ki]
print('Глобальный вектор нагрузок RG',RG)
RGpr = np.zeros((npe,m1))
ii = 0
for i in range(0, ng1, m1):
    for k in range(m):
        RGpr[ii,k] = RG[i+k-1]
    ii += 1
v = np.linalg.inv(KG) @ RG
vpr = np.zeros((npe,m1))
ii = 0
for i in range(0, ng1, m1):
    for k in range(m+1):
        vpr[ii, k] = -v[i+k]
    ii += 1
print('Узловые прогибы и углы поворота\n', vpr)

def vx (k, z, IE, v, Ag, Dh, ne, n1, m1, hi):
    vi = np.zeros((n1,1))
    zi = np.ones((n1,1))
    for i in range(1, n1):
        zi[i] = zi[i-1] * z / i
    j = 0
    i_start = m1*(IE)
    i_fin = i_start + n1
    for i in range(i_start, i_fin):
        vi[j] = v[i]
        j += 1
    Hvk = np.diag(np.eye((n1-k)))
    Hk = np.diag(Hvk,k)
    alpha = (Hk@Ag) @ (Dh@vi)
    vz= alpha.T@zi / hi**k
    return vz

d2v = np.zeros((ne,1))
for IE in range(0, ne):
    d2v[IE] = EJ * vx(2, 0.5, IE, v, Ag, Dh, ne, n1, m1, hi)
print('Изшибающие моменты в (в центрах элементов)\n',d2v.T)
d3v = np.zeros((ne,1))
for IE in range(0, ne):
    d3v[IE] = EJ * vx(3, 0.5, IE, v, Ag, Dh, ne, n1, m1, hi)

Глобальная матрица жесткости KG
 [[ 2632.86   216.96 -2577.86   215.43     0.       0.       0.       0.       0.       0.
      0.       0.       0.       0.  ]
 [  216.96    24.03  -215.43    11.98     0.       0.       0.       0.       0.       0.
      0.       0.       0.       0.  ]
 [-2577.86  -215.43  5265.71     0.   -2577.86   215.43     0.       0.       0.       0.
      0.       0.       0.       0.  ]
 [  215.43    11.98    -0.      48.06  -215.43    11.98     0.       0.       0.       0.
      0.       0.       0.       0.  ]
 [    0.       0.   -2577.86  -215.43  5265.71     0.   -2577.86   215.43     0.       0.
      0.       0.       0.       0.  ]
 [    0.       0.     215.43    11.98    -0.      48.06  -215.43    11.98     0.       0.
      0.       0.       0.       0.  ]
 [    0.       0.       0.       0.   -2577.86  -215.43  5265.71     0.   -2577.86   215.43
      0.       0.       0.       0.  ]
 [    0.       0.       0.       0.     215.43    11.98    -0.